# 1.1 Create unique points at junctions

***GeoPackages only allow a single geometry column in a table***

## 0. Libraries and settings

In [ ]:
# Libraries
from datetime import datetime
import geopandas as gpd
import pandas as pd
import shapely

#Settings
gpd.options.io_engine = "pyogrio"

print("Last run:", datetime.now())
print("GeoPandas version:", gpd.__version__)
print("Pandas version:", pd.__version__)
print("Shapely version:", shapely.__version__)

In [ ]:
lines_in = '../0_data/intermediate/4_lines.gpkg'
points_out = '../0_data/intermediate/4_lines_points.gpkg'

## 1. Read in data

### 1.1 lines

In [ ]:
lines = gpd.read_file( lines_in )

In [ ]:
lines.plot()
lines

## 2. Extract both start and end points 

In [ ]:
start_points = shapely.get_point(lines.geometry,0)

In [ ]:
end_points = shapely.get_point(lines.geometry,-1)

In [ ]:
all_points = pd.concat([start_points, end_points])

In [ ]:
all_points

## 3. Find unique points

In [ ]:
unique_points = all_points.drop_duplicates()
unique_points = unique_points.reset_index(drop=True) # Ensure ids are unique
unique_points

## 4. Convert to GeoDataFrame

In [ ]:
points = gpd.GeoDataFrame(geometry=unique_points, crs=lines.crs)

In [ ]:
points = points.reset_index(names='point_id')
points['point_id'] = 'p' + points['point_id'].astype('str')
points

In [ ]:
ax = lines.plot(color='lightgrey', zorder=-1)
points.plot(ax=ax, color='red', zorder=0)

# labels
for x, y, label in zip(points.geometry.x, points.geometry.y, points['point_id']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
for x, y, label in zip(lines.geometry.centroid.x, lines.geometry.centroid.y, lines['line_id']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

### Check point and line ids are unique

In [ ]:
lines['line_id'].is_unique

In [ ]:
points['point_id'].is_unique

## 4. Save Points

***Note: Lines are unchanged***

In [ ]:
points.to_file( points_out )